In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy import text
import settings
import time
import numpy as np
import pickle
import json
import glob



def db_connect():
    """
    Performs database connection using database settings from settings.py.
    Returns sqlalchemy engine instance
    """
    return create_engine(URL(**settings.DATABASE))

db = db_connect()

ImportError: C extension: /home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/hashtable.so: failed to map segment from shared object: Cannot allocate memory not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace' to build the C extensions first.

In [ ]:
#build new SQL tables

#get list of all tables in db
from sqlalchemy import MetaData
start_time = time.time()
m = MetaData(bind=db)
m.reflect()
tables = list(m.tables.keys())
time.time()-start_time

# from list of tables, get all w_chartevent tables
import re
tab_re = re.compile(r'^(w_chartevents_.*)')
chart_tables = []
for table in tables:
    name=re.findall(tab_re, table)
    if name:
        chart_tables.append(name[0])
w_chart_tables = sorted(chart_tables)

In [ ]:
#combine pickles (w_chart_tabes = list of tables you want to join and work on )
df = combine_pickles(w_chart_tables)

In [ ]:
#get what items to cut
keep_items = pd.read_csv('../TextFiles/Keep_items.txt', names = ['itemsid'])
keep_list = list(keep_items.itemsid.values)
df.reset_index(drop=True, inplace=True)


In [ ]:
#drop bad items
count = 0
total = len(keep_list)
keep_index = df[df.itemid==keep_list[0]].index
for itemid in keep_list[1:]:
    keep_index = keep_index.append(df[df.itemid==itemid].index)
    count +=1
    print "\r",float(count)/total * 100, "% done",
print 'final step...'
drop_index = df.index.difference(keep_index)
df.drop(drop_index, axis = 0, inplace=True)
print 'done!'

In [ ]:
#map items
def items_to_item(df):
    '''
    FUNCTION: map old item IDs to new item ID
    INPUT = data frame
    OUTPUT = data frame with new item IDs in place
    '''
    from collections import defaultdict
    import json
    items_dict = defaultdict(list)

    with open("../JSONs/items_to_item.json", 'r') as f:
        items_dict = json.load(f)
    items_dict = {int(k):int(v) for k,v in items_dict.items()}
    df['itemid'].replace(items_dict, inplace = True)
    return df

df = items_to_item(df)

In [17]:
#build discharge time - icustayid dictionary
#get icustay table
que = '''select * from icustays'''
icustays_df = pd.read_sql(que, db)

icustays_df = icustays_df[['icustay_id', 'intime','outtime']]

In [18]:
distime_dict = {}
intime_dict = {}
for icustay_id in df.icustay_id.unique():
    distime_dict[icustay_id] = icustays_df[icustays_df.icustay_id == icustay_id].outtime.values[0]
    intime_dict[icustay_id] = icustays_df[icustays_df.icustay_id == icustay_id].intime.values[0]

In [21]:
#flatten chart
a = flatten_wchart(df, dt_dict)

NameError: name 'dt_dict' is not defined

In [ ]:
#rename columns
#get itemid
d_items_df = pd.read_pickle('../Pickles/d_items.p')
#need to create dictionary with only needed itemids

df_items = df.columns.drop('icustay_id')
itn_dict = {}
for item in df_items:
    itn_dict[item] = d_items_df[d_items_df.itemid == item].label.values[0]

In [ ]:
a.rename(columns = itn_dict, inplace=True)

In [ ]:
#store flattened df to pickle files
#length = len(a)
for i in range(4):
    start_time = time.time()
    filename = '../Pickles/flat_chart_' + str(i) + '.p'
    end = (i+1)*length/4
    start = (i)*length/4
    a[start:end].to_pickle(filename)
    print '\r', time.time()-start_time, filename, 'done',

In [8]:
#reading from pickle files
import glob
files = glob.glob('../Pickles/flat_*')
dflist = []
for filename in files:
    print '\r', filename
    dflist.append(pd.read_pickle(filename))

df = pd.concat(dflist)
dflist = None
print "Done!"

../Pickles/flat_chart_0.p
../Pickles/flat_chart_2.p
../Pickles/flat_chart_1.p
../Pickles/flat_chart_3.p
Done!


In [9]:
#for each itemid, look at how many empty cells there are 
cols = list(df.drop('Code Status', axis = 1).columns)
cols.remove('icustay_id')
null_list = []
for col in cols:
    null_list.append([col, pd.isnull(df[col]).sum()])

null_list.sort(key = lambda x:x[1])

In [10]:
#build target
patient_df = pd.read_csv('../TextFiles/FINAL_patient_list.csv', index_col = 0)
df = df.merge(patient_df[['icustay_id', 'hospital_expire_flag']], on='icustay_id')

In [12]:
features = [x[0] for x in null_list[:]]
features.append('icustay_id')
features.append('hospital_expire_flag')

In [97]:
cols = list(df.columns)
cols.remove('icustay_id')
cols.remove('hospital_expire_flag')

In [98]:
#get data from 24 hours after admit
start_time = time.time()
model_df = df.apply(post_admit_row, cols = cols, axis=1)
print time.time()-start_time

model_df.to_pickle('0810_24hour_post_admit_allfeat.p')

1501.37234902


In [218]:
#get time series data from vital, 24 hours after admit

vit_cols = ['Arterial BP [Systolic]',
 'Heart Rate',
 'Arterial BP [Diastolic]',
 'Resp Rate (Total)',
 'SpO2',
 'Temperature Fahrenheit',
 'icustay_id']

vit_df = df[vit_cols]
cols = vit_cols[:]
cols.remove('icustay_id')

start_time = time.time()
vit_model_df = vit_df.apply(post_admit_row_trans, cols = cols, axis=1)
print time.time()-start_time

vit_model_df.to_pickle('0811_trans_vit_df.p')

In [232]:
cols = list(df.columns)
cols.remove('icustay_id')
cols.remove('hospital_expire_flag')

#get data from 24 hours from death
start_time = time.time()
pre_death_df = df.apply(before_death_row, cols = cols, axis=1)
print time.time()-start_time
pre_death_df.to_pickle('0811_PD_model_df.p')

1507.77364993


In [233]:
#get vital transforms for 24 hours from death
vit_df = df[vit_cols]
cols = vit_cols[:]
cols.remove('icustay_id')

start_time = time.time()
pd_vtrans_df = vit_df.apply(before_death_row_trans, cols = cols, axis=1)
print time.time()-start_time

pd_vtrans_df.to_pickle('0811_PD_trans_model_df.p')

1194.6721549


In [12]:
def combine_pickles(tables):
    start_time = time.time()
    dflist = []
    count = 1
    for table in tables:
        print "\r","working on %s, %d/%d tables" % (table, count, len(tables)),
        filename = "../Pickles/" + table + '.p'
        dflist.append(pickle.load(open(filename, 'rb')))
        count +=1
    df = pd.concat(dflist)
    dflist=None
    print time.time()-start_time
    return df

In [13]:
def flatten_wchart(df, dt_dict):
    '''
    after getting all w_charts into pandas:
    build features!
    '''
    print 'starting up...', 
    start_time = time.time()
    count = 0
    row_list = []
    keep_columns = ['charttime', 'value', 'valuenum', 'valueuom', 'icustay_id', 'itemid']
    df = df[keep_columns]
    
    grouped_chart = df.groupby('icustay_id')
    total = len(grouped_chart.groups.keys())
    
    for patient in grouped_chart.groups.keys():
        row_dict={}
        pat_df = grouped_chart.get_group(patient)
        group_item = pat_df.groupby('itemid')
        row_dict['icustay_id'] = patient
        distime = dt_dict[patient]
        for itemid in group_item.groups.keys():
            item_df = group_item.get_group(itemid)
            item_df = item_df[(distime - item_df.charttime) > pd.Timedelta('1 day')]
            if len(item_df) < 1:
                row_dict[itemid] = np.nan
            else:
                row_dict[itemid] = item_df.sort_values(by='charttime', ascending=False).values.tolist()
        row_list.append(row_dict)
        count += 1 #print progress
        perc = float(count)/total * 100
        print '\r', perc, 
        
    features_df = pd.DataFrame(row_list)
    row_list = None
    row_dict = None
    grouped_chart = None
    
    return features_df

In [4]:
def get_wchart_data(tables, db):
    '''Gets all tables in list from sql server, loads them into memory and stores as .pickle'''
    for table in tables:
        filename = table + '.p'
        start_time = time.time()
        command =  '''select * from %s''' % table
        sql = text(command)
        temp_df = pd.read_sql(sql, db)
        print(table)
        print(time.time()-start_time)
        print "Writing %s to pickle... %s" % (table, filename)
        filename = table + '.p'
        start_time = time.time()
        pickle.dump(temp_df, open(filename , "wb" ) )
        print "Done writing pickle", time.time()-start_time
    return 'Done'

In [5]:
def get_most_recent(entries):
    '''Sort by time, return most recent data '''
    try:
        sorted_entries = sorted(entries, key = lambda x: x[0])
        return sorted_entries[-1][2]
    except:
        return np.nan

In [206]:
import time
import sys
def before_death_row(df_row, cols, hours=24):
    row = {}
    admit_lag = str(hours) + " Hours"
    row['hospital_expire_flag'] = df_row.loc['hospital_expire_flag']
    row['icustay_id'] = df_row.loc['icustay_id']
    distime = pd.Timestamp(distime_dict[row['icustay_id']])
    #print intime
    for col in cols:
        try:
            data = [x[2] for x in df_row.loc[col] if distime - x[0] > pd.Timedelta(admit_lag)]
            if len(data) < 1:
                data = np.nan
            row[col] = data[0]
        except:
            row[col] = np.nan
    result_df = pd.Series(row)
    return result_df
    

In [230]:
def before_death_row_trans(df_row, cols, hours=24):
    row = {}
    admit_lag = str(hours) + " Hours"
    row['icustay_id'] = df_row.icustay_id
    distime = pd.Timestamp(distime_dict[row['icustay_id']])
    #print intime
    for col in cols:
        median = col + '_median'
        std = col + '_std'
        rng = col + '_range'
        try:
            data = [x[2] for x in df_row.loc[col] if (distime - x[0] > pd.Timedelta(admit_lag)) & (distime - x[0] < 2*pd.Timedelta(admit_lag))]
            if len(data) < 1:
                data = np.nan
            row[col] = data[0]
            row[median] = np.median(data)
            if len(data) > 2:
                row[std] = np.std(data)
                row[rng] = np.ptp(data)
        except:
            row[col] = np.nan
    result_df = pd.Series(row)
    return result_df

In [164]:
def post_admit_row(df_row, cols, hours=24):
    row = {}
    admit_lag = str(hours) + " Hours"
    row['hospital_expire_flag'] = df_row.loc['hospital_expire_flag']
    row['icustay_id'] = df_row.loc['icustay_id']
    intime = intime_dict[row['icustay_id']]
    #print intime
    for col in cols:
        try:
            data = [x[2] for x in df_row.loc[col] if x[0] - intime < pd.Timedelta(admit_lag)]
            if len(data) < 1:
                data = np.nan
            row[col] = data[0]
        except:
            row[col] = np.nan
    result_df = pd.Series(row)
    return result_df

In [157]:
def post_admit_row_trans(df_row, cols, hours=24):
    row = {}
    admit_lag = str(hours) + " Hours"
    row['icustay_id'] = df_row.icustay_id
    intime = intime_dict[row['icustay_id']]
    #print intime
    for col in cols:
        median = col + '_median'
        std = col + '_std'
        rng = col + '_range'
        try:
            data = [x[2] for x in df_row.loc[col] if x[0] - intime < pd.Timedelta(admit_lag)]
            if len(data) < 1:
                data = np.nan
            row[col] = data[0]
            row[median] = np.median(data)
            if len(data) > 2:
                row[std] = np.std(data)
                row[rng] = np.ptp(data)
        except:
            row[col] = np.nan
    result_df = pd.Series(row)
    return result_df